# Ollama

In [1]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

MODEL_NAME = "llama3.1:latest"

OLLAMA_URL = OpenAIProvider(
    base_url="http://localhost:11434/v1"
)

OLLAMA_MODEL = OpenAIModel(
    model_name=MODEL_NAME,
    provider=OLLAMA_URL
)

# Agents

In [2]:
from pydantic_ai.agent import Agent
    
recepcionist_agent = Agent(
    model=OLLAMA_MODEL,
    instructions=
    """You are Tera, a recepcionist for Stray Animal Care, an animal care clinic specialized in cats and dogs.
    Your job is to take the client's name, pet name and species, along with symptoms (if there are) and according to that info, 
    refer it to either Dr. Doug Dogson, the dog specialist or to Kathy Katrina, the cat specialist.""",
    output_type=str,
    model_settings={"temperature": 0.9},
    retries=3,
)

dog_veterinarian = Agent(
    model=OLLAMA_MODEL,
    instructions=
    """
    You are Dr. Doug Dogson, a veterinarian specialized in dogs working at the Stray Animal Care, an animal care clinic specialized in cats and dogs. 
    Greet the user and it's pet in a professional way. Use a few puns to make the convo friendly. 
    According to the intake form, deal with the issue at hand.
    """,
    output_type=str,
    model_settings={"temperature": 0.6},
    retries=3
)

cat_veterinarian = Agent(
    model=OLLAMA_MODEL,
    instructions=
    """
    You are Dr. Cathy Catrina, a veterinarian specialized in dogs working at the Stray Animal Care, an animal care clinic specialized in cats and dogs. 
    Greet the user and it's pet in a professional way. Use a few puns to make the convo friendly. 
    According to the intake form, deal with the issue at hand.
    """,
    output_type=str,
    model_settings={"temperature": 0.6},
    retries=3
)

### Tools

In [3]:
from pydantic import BaseModel, Field
from typing import Optional, Literal

class IntakeForm(BaseModel):
    """A structured form containing all necessary information from a new client's request."""
    client_name: str = Field(description="The name of the pet's owner.")
    pet_name: str = Field(description="The name of the pet.")
    pet_species: Literal["cat", "dog"] = Field(description="The type of animal. Must be one of 'cat' or 'dog'.")
    symptoms: Optional[str] = Field(description="A summary of the pet's symptoms.")

@recepcionist_agent.tool_plain
async def call_vet(args: IntakeForm) -> str:
    """Takes structured client info from the IntakeForm and routes the case to the correct specialist agent."""

    specialist_response = ""
    if args.pet_species == "dog":
        prompt_for_dog_vet = f"""A new case has been assigned to you.
        -Client's name: {args.client_name}
        -Dog's name: {args.pet_name}
        -Reported symptoms: {args.symptoms}"""

        dog_vet_analysis = await dog_veterinarian.run(prompt_for_dog_vet)
        specialist_response = dog_vet_analysis.output

    elif args.pet_species == "cat":
        prompt_for_cat_vet = f"""A new case has been assigned to you.
        -Client's name: {args.client_name}
        -Cat's name: {args.pet_name}
        -Reported symptoms: {args.symptoms}"""

        cat_vet_analysis = await cat_veterinarian.run(prompt_for_cat_vet)
        specialist_response = cat_vet_analysis.output
    
    return f"SYSTEM: The specialist has responded. Your ONLY job now is to output the specialist's response verbatim. Here it is: '{specialist_response}'"


### tools

In [5]:
from rich.console import Console
console = Console(record=True)

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        result = await recepcionist_agent.run(user_input)
    except:
        print("ERROR")
        break
        
        

# console.print(result.all_messages())
# print(result.output)


Goodbye!
